In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os

# Configuration
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 10 
DATASET_DIR = "PlantVillage"

# Check for GPU availability (Optional)
print(f"TensorFlow Version: {tf.__version__}")
print(f"Dataset Path: {os.path.abspath(DATASET_DIR)}")

In [ ]:
# Load Dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

class_names = dataset.class_names
print(f"Classes found: {class_names}")

# Visualize samples
plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
# Pipeline optimitation
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# CNN Architecture
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.Rescaling(1./255)
])

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Train
print("Starting training...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

In [ ]:
# --- CELDA 4: Visualización de Resultados ---

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Loss
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

# --- Quick Prediction Test ---
import numpy as np

print("\n--- Quick Prediction Test ---")
for images_batch, labels_batch in test_ds.take(1):
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    plt.imshow(first_image)
    plt.axis('off')
    
    # Run prediction
    batch_prediction = model.predict(images_batch)
    prediction = batch_prediction[0]
    predicted_class = class_names[np.argmax(prediction)]
    confidence = round(100 * (np.max(prediction)), 2)
    
    actual_class = class_names[first_label]
    
    plt.title(f"Actual: {actual_class}\nPrediction: {predicted_class} ({confidence}%)")
    plt.show()
    break

In [ ]:
# --- Save Model ---
import os

# CCreate model directory if it doesn't exist
if not os.path.exists('modelos'):
    os.makedirs('modelos')

# Save model in .keras format
model_version = 1
model.save(f"modelos/potato_model_v{model_version}.keras")

print("Model saved successfully to 'modelos' directory!")

In [ ]:
# --- Test Prediction ---
import numpy as np

# TGet a batch of images from the test set
for images_batch, labels_batch in test_ds.take(1):
    
    # Select the first image from the batch
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    # Show image
    plt.imshow(first_image)
    plt.axis('off')
    
    # Run prediction
    # The model expects a batch, so we expand dimensions
    batch_prediction = model.predict(np.expand_dims(images_batch[0], axis=0))
    
    # Get the class with the highest probability
    pred_class_index = np.argmax(batch_prediction[0])
    predicted_class = class_names[pred_class_index]
    confidence = round(100 * (np.max(batch_prediction[0])), 2)
    
    actual_class = class_names[first_label]
    
    print(f"Actual Class: {actual_class}")
    print(f"Predicted Class:    {predicted_class}")
    print(f"Confidence:        {confidence}%")
    
    plt.title(f"Pred: {predicted_class} ({confidence}%)")
    plt.show()
    break # Only one